In [14]:
import numpy as np
import pandas as pd

recording_path = "../recordings/"
output_path = "../trim_output/"
file_name = input("Input File Name")

#Read in CSV

try:
    dataframe = pd.read_csv(recording_path + file_name + ".csv")
except:
    print("Error Reading File: Check Spelling and Try Again")

# Seperate each tracker to seperate dataframe
HMD = dataframe.iloc[:, 1:4]
controller_1 = dataframe.iloc[:, 7:10]
controller_2 = dataframe.iloc[:, 37:40]
left_foot = dataframe.iloc[:, 67:70]
right_foot = dataframe.iloc[:, 73:76]
waist = dataframe.iloc[:, 79:82]

# Join all trackers together
joined = pd.concat([HMD, controller_1, controller_2,
                   waist, left_foot, right_foot], axis=1)

# set new column headers
joined.columns = [
    "head_x",
    "head_y",
    "head_z",
    "r_controller_x",
    "r_controller_y",
    "r_controller_1_z",
    "l_controller_x",
    "l_controller_y",
    "l_controller_z",
    "waist_x",
    "waist_y",
    "waist_z",
    "r_foot_x",
    "r_foot_y",
    "r_foot_z",
    "l_foot_x",
    "l_foot_y",
    "l_foot_z"
]


# output to new csv
output_file = output_path + file_name + "_trimmed.csv"
joined.to_csv(output_file, index=False)

print(file_name + " output to " + output_path)


Error Reading File: Check Spelling and Try Again


ValueError: Length mismatch: Expected axis has 6 elements, new values have 18 elements

In [16]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


output_path = "../trim_output/"



file_name = input("Input File Name")
dataframe = pd.read_csv(output_path + file_name + ".csv")
dataframe.describe()
arr = np.array(dataframe)

#divide array by 100 to normalize to original values
array = np.divide(arr, 100)

#round to nearest 3 points
roundedArray = np.around(array, 3)
df = pd.DataFrame(roundedArray)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-0.015,1.572,-0.099,0.212,0.792,-0.114,-0.261,0.769,-0.101,-0.004,0.969,-0.085,0.234,0.095,0.015,-0.207,0.093,0.006
1,-0.015,1.572,-0.099,0.211,0.791,-0.114,-0.260,0.770,-0.102,-0.004,0.969,-0.085,0.235,0.095,0.015,-0.207,0.093,0.006
2,-0.015,1.572,-0.099,0.211,0.791,-0.113,-0.259,0.770,-0.102,-0.005,0.970,-0.085,0.235,0.095,0.015,-0.207,0.093,0.006
3,-0.015,1.572,-0.099,0.210,0.791,-0.113,-0.259,0.770,-0.102,-0.005,0.969,-0.085,0.234,0.096,0.015,-0.206,0.093,0.006
4,-0.015,1.572,-0.099,0.210,0.791,-0.113,-0.258,0.770,-0.102,-0.005,0.970,-0.085,0.234,0.096,0.015,-0.206,0.093,0.006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,-0.028,1.604,-0.085,0.242,0.840,-0.164,-0.193,1.172,-0.256,-0.013,0.977,-0.098,0.235,0.097,0.016,-0.205,0.096,0.007
901,-0.028,1.604,-0.085,0.242,0.840,-0.164,-0.193,1.172,-0.256,-0.012,0.977,-0.098,0.235,0.097,0.016,-0.205,0.096,0.007
902,-0.028,1.604,-0.085,0.241,0.840,-0.164,-0.194,1.172,-0.256,-0.013,0.977,-0.099,0.235,0.097,0.016,-0.205,0.096,0.007
903,-0.028,1.604,-0.085,0.241,0.840,-0.164,-0.193,1.172,-0.256,-0.013,0.977,-0.099,0.235,0.097,0.016,-0.205,0.096,0.007


In [7]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
model = keras.Sequential()
model.add(layers.SimpleRNN(64, input_shape=(None, 28)))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10))
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 64)                5952      
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 6,858
Trainable params: 6,730
Non-trainable params: 128
_________________________________________________________________
None
